In [1]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle, islice
%matplotlib inline  
import warnings
warnings.filterwarnings('ignore')
from people import people


In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
from IPython.display import display, HTML
def pretty_print(df):
    return display(HTML(df.to_html().replace("\n","<br>")))

In [9]:
results_file = '%s_results_wup_0.76.csv'
df = pd.read_csv(results_file % ('ivan'))

# remove rows where no clusters
df['Ivan_clusters'] = df['human_clusters']
df['Ivan_hits'] = df['hits']
df['Ivan_success'] = df['hits'] > 0
df = df.dropna(subset=['human_clusters'])


for person in set(people).difference('Ivan'):
    df_tmp = pd.read_csv(results_file % (person))
    df['%s_clusters' % (person)] = df_tmp['human_clusters']
    df['%s_hits' % (person)] = df_tmp['hits']
    df['%s_success' % (person)] = df_tmp['hits'] > 0

df=df.fillna('')

# How well overall did machine do

T = 0.8, similarity metric is how similar two word senses are, based on the depth of the two senses in the taxonomy and that of their Least Common Subsumer (most specific ancestor node).

In [10]:
columns = ['artist_name', 'human_clusters', 'human_assessment_type', 'machine_clusters'] + ['%s_hits' % (person) for person in people]
df_summary = df[columns]

def custom_style(row):
    color = 'white'
    if row['Vincent_hits'] == 3: color = '#FFFF99' 
    if row['Vincent_hits'] == 2: color = '#FFFF00' 
    if row['Vincent_hits'] == 1: color = '#FFFF66' 
    return ['background-color: %s' % color]*len(row.values)

df_summary.style.apply(custom_style, axis=1)

,artist_name,human_clusters,human_assessment_type,machine_clusters,Ivan_hits,Vincent_hits,Marie_hits,Kathleen_hits
0,Du Zhenjun,"Power,Time",tags,"Time,Power,Arts",2,1,1,0
1,Jiang Pengyi,Fantasy,tags,"Fantasy,Arts,Land",1,1,1,1
2,Eleanor Antin,Inequality,tags,"Arts,Activism,Geopolitics",0,0,0,0
3,Mathilde Rosier,"Emotion,Fantasy",tags,"Materiality,Emotion,Geopolitics",1,0,1,1
4,Brenna Youngblood,Immaterial,tags,"Land,Time,Space",0,0,0,1
5,Dominique Zinkpè,"Body,Spirituality",tags,"Spirituality,Body,Identity",2,1,1,3
6,Hans Peter Feldmann,Materiality,tags,Immaterial,0,0,0,1
7,Walead Beshty,Materiality,tags,"Arts,Materiality,Land",1,1,1,1
8,Miri Segal,"Body,Technology",tags,"Mass media,Culture,Technology",1,1,1,0
9,Diana Thater,"Physics,Mind",tags,"Geopolitics,Body,Environmental",0,1,1,1


# How well did machine do by human assessment type

In [11]:
x = df[['human_assessment_type', '%s_clusters' % ('Vincent'), 'machine_clusters', 'hits']]    
x.groupby('human_assessment_type').agg({'hits':sum}).head()


,hits
human_assessment_type,
description,19
tags,22


# Now calculate the multi-label metric

The Hamming score is closely related to the Hamming loss (the fraction of labels that are incorrectly predicted), but allows for different number of labels (clusters) predicted by man and machine.

In [149]:
from metrics import hamming_score
for person in people:
    y_true = list(df['%s_clusters' % (person)].str.split(','))
    y_pred = list(df['machine_clusters'].str.split(','))
    score = hamming_score(y_true, y_pred)
    print('{0} <--> computer score \t{1}'.format(person, score))
    
    

Ivan <--> computer score 	0.6950354609929079
Vincent <--> computer score 	0.9432624113475176
Marie <--> computer score 	0.9432624113475176
Kathleen <--> computer score 	0.9219858156028368


In [153]:
from metrics import hamming_score
anchor = 'Kathleen'
for person in set(people) - set([anchor]):
    y_true = list(df['%s_clusters' % (person)].str.split(','))
    y_pred = list(df['%s_clusters' % (anchor)].str.split(','))
    score = hamming_score(y_true, y_pred)
    print('{0} <--> {1} \t{2}'.format(person, anchor, score))


Marie <--> Kathleen 	0.9716312056737588
Vincent <--> Kathleen 	0.9787234042553191
Ivan <--> Kathleen 	0.6666666666666667
